# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Контрольная работа

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk


file_path = 'preprocessed_descriptions.csv'
df = pd.read_csv(file_path)


descriptions = df['description'].tolist()


unique_words = set()

for description in descriptions:
    tokens = word_tokenize(description)
    unique_words.update(tokens)


print(unique_words)


nltk.download('punkt')


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import random
from nltk.metrics import edit_distance

nltk.download('punkt')

file_path = 'preprocessed_descriptions.csv'
df = pd.read_csv(file_path)

descriptions = df['description'].tolist()

unique_words = set()

for description in descriptions:
    tokens = word_tokenize(description)
    unique_words.update(tokens)

unique_words_list = list(unique_words)

random_pairs = [(random.choice(unique_words_list), random.choice(unique_words_list)) for _ in range(5)]

distances = [(word1, word2, edit_distance(word1, word2)) for word1, word2 in random_pairs]

for word1, word2, dist in distances:
    print(f"Слова: {word1}, {word2} -> Расстояние редактирования: {dist}")


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [5]:
import heapq
from nltk.metrics import edit_distance
from typing import List, Tuple

def k_nearest_words(word: str, words: List[str], k: int) -> List[Tuple[str, int]]:
    """
    Возвращает k ближайших слов к заданному слову из списка, используя расстояние Левенштейна.

    :param word: Заданное слово
    :param words: Список слов
    :param k: Количество ближайших слов, которые нужно вернуть
    :return: Список из k ближайших слов и их расстояний в виде кортежей (слово, расстояние)
    """
    
    distances = [(w, edit_distance(word, w)) for w in words]
    
    
    k_nearest = heapq.nsmallest(k, distances, key=lambda x: x[1])
    
    return k_nearest

# Пример со словами
word = "cake"
words = ["bake", "make", "lake", "rake", "take", "fake", "stake", "snack", "break", "quake"]
k = 3

nearest_words = k_nearest_words(word, words, k)
print(nearest_words)


[('bake', 1), ('make', 1), ('lake', 1)]


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

file_path = 'preprocessed_descriptions.csv'
df = pd.read_csv(file_path)

descriptions = df['description'].tolist()

unique_words = set()
for description in descriptions:
    tokens = word_tokenize(description)
    unique_words.update(tokens)

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

data = []
for word in unique_words:
    stemmed_word = stemmer.stem(word)
    normalized_word = lemmatizer.lemmatize(word)
    data.append((word, stemmed_word, normalized_word))

df_words = pd.DataFrame(data, columns=['word', 'stemmed_word', 'normalized_word'])
df_words.set_index('word', inplace=True)

print(df_words.head())

for i in range(5):
    print(f"Word: {df_words.index[i]}, Stemmed: {df_words.iloc[i, 0]}, Lemmatized: {df_words.iloc[i, 1]}")



2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).